In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade



You should consider upgrading via the '/Users/elmonoloco/anaconda3/envs/PythonData/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.2.0

Found existing installation: tensorflow 2.2.0
Uninstalling tensorflow-2.2.0:
  Successfully uninstalled tensorflow-2.2.0
  Using cached tensorflow-2.2.0-cp36-cp36m-macosx_10_11_x86_64.whl (175.3 MB)
You should consider upgrading via the '/Users/elmonoloco/anaconda3/envs/PythonData/bin/python -m pip install --upgrade pip' command.


In [1]:

import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense

In [2]:
# Read the CSV and Perform Basic Data Cleaning
df = pd.read_csv("mushrooms3.csv")
# Drop the null columns where all values are null
#df = df.dropna(axis='columns', how='all')
# Drop the null rows
#df = df.dropna()
df.head()

,class,cap-shape,cap-surface,cap-color,odor,spore-print-color,population
0,1,2,3,0,7,0,3
1,0,2,3,9,0,1,2
2,0,0,3,8,1,1,2
3,1,2,2,8,7,0,3
4,0,2,3,3,6,1,0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [4]:
#Create a Train Test Split
y = df["class"]
X = df.drop(columns=["class"])
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

(8124, 6) (8124,)


In [5]:
import os
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
import pandas as pd
import numpy as np
from flask import Flask, jsonify, render_template, request
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [6]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [7]:
y_train_categorical.shape

(6093, 2)

In [8]:
# Pre-processing
# Scale your data

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
print(X_test[:20])

      cap-shape  cap-surface  cap-color  odor  spore-print-color  population
5525          2            2          5     6                  7           1
5868          2            2          0     8                  7           4
5111          3            2          9     4                  3           4
3193          3            0          9     4                  3           4
1561          3            3          3     6                  0           3
3603          3            2          7     6                  0           4
2360          2            2          0     6                  1           5
923           2            3          9     1                  6           4
2690          3            3          0     6                  1           0
612           3            3          8     6                  0           0
5607          3            2          9     4                  3           4
1237          3            0          3     6                  1           3

In [10]:
#train the model
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=6))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [11]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
# set early stopping as callback
from tensorflow.keras.callbacks import EarlyStopping
callbacks = [EarlyStopping(monitor='loss', patience=2)]
model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
191/191 - 0s - loss: 0.0053 - accuracy: 0.9974
Epoch 2/60
191/191 - 0s - loss: 0.0069 - accuracy: 0.9972
Epoch 3/60
191/191 - 0s - loss: 0.0055 - accuracy: 0.9979


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               700       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 11,002
Trainable params: 11,002
Non-trainable params: 0
_________________________________________________________________


In [22]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

64/64 - 0s - loss: 0.0142 - accuracy: 0.9961
Normal Neural Network - Loss: 0.01421726681292057, Accuracy: 0.9960610270500183


In [23]:
encoded_predictions = model.predict_classes(X_test_scaled[:6])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [24]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:6])}")

Predicted classes: [0 1 1 1 0 0]
Actual Labels: [0, 1, 1, 1, 0, 0]


In [25]:
# Load the model
from tensorflow.keras.models import load_model

# save fitted model to file
model.save("mushrooms_deeplearn2.h5")